Leer Texto de imagenes

In [1]:
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
import csv
import matplotlib.pyplot as plt
import cv2
import os
import torch

In [ ]:
# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Definir la ruta al archivo CSV
csv_file = r'..\..\data\output\metadata.csv'

# Función para obtener recursivamente todos los archivos de imágenes
def list_files_recursive(folder_path):
    image_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg'):
                image_files.append(os.path.join(root, file))
    return image_files

# Clase Dataset personalizada para cargar datos desde el archivo CSV
class CustomDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = pd.read_csv(csv_file)
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        image_path = self.data.iloc[idx, 0]  # primera columna es image_dir
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convertir a RGB

        text = self.data.iloc[idx, 2]  # tercera columna es text

        if self.transform:
            image = self.transform(image)

        return image, text

# Transformaciones de imagen
transform = transforms.Compose([
    transforms.ToTensor(),
])

# Dataset y DataLoader
dataset = CustomDataset(csv_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=1, shuffle=True)

# Cargar el procesador y el modelo TrOCR
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten').to(device)

# Función para entrenar el modelo
def train_model(model, dataloader, num_epochs=5):
    model.train()
    for epoch in range(num_epochs):
        print(f'Epoch {epoch + 1}/{num_epochs}')
        for images, texts in dataloader:
            images = images.to(device)
            with torch.no_grad():
                pixel_values = processor(images=images, return_tensors="pt").pixel_values.to(device)
            generated_ids = model.generate(pixel_values)
            generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

            # Mostrar la imagen y el texto reconocido
            plt.imshow(images.squeeze(0).permute(1, 2, 0).cpu().numpy())
            plt.title('Texto reconocido: ' + generated_text)
            plt.axis('off')
            plt.show()

            # Escribir el texto reconocido en la consola
            print(f'Imagen: {images.shape}')
            print(f'Texto original: {texts}')
            print(f'Texto reconocido: {generated_text}')
            print('---')

# Entrenar el modelo
train_model(model, dataloader)

In [2]:
# Verificar si CUDA está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# Obtener la lista de archivos en la carpeta y subcarpetas
folder_path = r'..\..\data\output\divide_images_test'

# Función para obtener recursivamente todos los archivos de imágenes
def list_files_recursive(folder_path):
    image_files = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith('.png') or file.endswith('.jpg') or file.endswith('.jpeg'):
                image_files.append(os.path.join(root, file))
    return image_files

image_files = list_files_recursive(folder_path)

# Cargar el procesador y el modelo
processor = TrOCRProcessor.from_pretrained('microsoft/trocr-large-handwritten')
model = VisionEncoderDecoderModel.from_pretrained('microsoft/trocr-large-handwritten').to(device)

# Iterar sobre cada imagen encontrada
for image_path in image_files:
    # Cargar la imagen
    image = cv2.imread(image_path)

    # Preprocesamiento de la imagen
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    _, binary_image = cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)

    # Convertir la imagen en escala de grises a una imagen con 3 canales
    binary_image_rgb = cv2.cvtColor(binary_image, cv2.COLOR_GRAY2RGB)

    # Convertir la imagen a tensor flotante de PyTorch y escalar los valores de píxeles
    image_tensor = torch.tensor(binary_image_rgb / 255.0, dtype=torch.float32).permute(2, 0, 1).unsqueeze(0).to(device)

    # Procesar la imagen y generar el texto
    pixel_values = processor(images=image_tensor, return_tensors="pt").pixel_values.to(device)
    generated_ids = model.generate(pixel_values)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]

    # Mostrar la imagen y el texto reconocido
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    plt.title('Texto reconocido: ' + generated_text)
    plt.axis('off')
    plt.show()

    # Escribir el texto reconocido en la consola
    print(f'Imagen: {image_path}')
    print(f'Texto reconocido: {generated_text}')
    print('---')

print("Procesamiento completo.")

Device: cpu


e:\ANACONDA\envs\SATURDAYSAI-TENSOR-CV2\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of VisionEncoderDecoderModel were not initialized from the model checkpoint at microsoft/trocr-large-handwritten and are newly initialized: ['encoder.pooler.dense.bias', 'encoder.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
e:\ANACONDA\envs\SATURDAYSAI-TENSOR-CV2\lib\site-packages\transformers\generation\utils.py:1133: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We 

KeyboardInterrupt: 